# Manipulación y preparación de datos

Tarea:
Realizar tareas de manipulación y preparación de datos utilizando herramientas y bibliotecas específicas.

Instrucciones:

Importar los conjuntos de datos con los que se trabajará.
Realizar tareas de limpieza de datos, eliminando valores nulos o inconsistentes.
Realizar transformaciones necesarias para preparar los datos para el análisis y modelado.

Importancia: Esta fase es crucial para asegurar que los datos estén en un formato adecuado y limpio, lo que es esencial para la construcción de modelos precisos y fiables.

#### Importamos las librerias necesarias para las tareas

In [155]:
import pandas as pd

#### Importamos los datos para trabajar con ellos

In [156]:
#Cargamos los datos de los archivos csv en dataframes de pandas
Mutations_df = pd.read_csv("./DataSet_Equipo17/TCGA_GBM_LGG_Mutations_all.csv")
info_df = pd.read_csv("./DataSet_Equipo17/TCGA_InfoWithGrade.csv")

#### Hacemos tecnicas de limpieza de datos para su futuro análisis

In [157]:
# Limpieza de datos
# Eliminar filas con valores nulos
Mutations_df.dropna(inplace=True)
# Pareciera que el dataset de info ya fue limpiado previamente pues todos los datos estan trabajados por lo que enfocare en limpiar el dataset de mutaciones
info_df.dropna(inplace=True)

In [158]:
# Reemplazar los valores de las columnas que  pueden ser tratados como datos binarios 
# como las columnas que informan de mutaciones, grado y genero por 0 o 1 para facilitar el análisis 
Mutations_df.replace({'NOT_MUTATED': 0, 'MUTATED': 1}, inplace=True)
Mutations_df.replace({'Female': 0, 'Male': 1}, inplace=True)
Mutations_df.replace({'LGG': 0, 'GBM': 1}, inplace=True)


In [159]:
# Crear diccionario de mapeo para la columna Race
race_mapping = {
    'not reported': 0,
    '--': 0,
    'white': 1,
    'black or african american': 2,
    'asian': 3,
    'american indian or alaska native': 4
}

# Aplicar el mapeo a la columna Race
Mutations_df['Race'] = Mutations_df['Race'].map(race_mapping)

In [160]:
# Función para convertir 'years days' a número decimal
def convertir_edad_decimal(age_str):
    try:
        if not "days" in age_str:
            return float(age_str.split(' years')[0])
        else:
            if ' years ' in age_str:
                years, days = age_str.split(' years ')
                years = int(years)
                days = int(days.split(' days')[0])
                return round(years + days / 365.25, 2)  # Redondear a 2 decimales
    except ValueError:
        # En caso de que no se tenga el valor de la edad, devolver None
        return None

# Aplicar la función a la columna 'Age_at_diagnosis'
Mutations_df['Age_at_diagnosis'] = Mutations_df['Age_at_diagnosis'].apply(convertir_edad_decimal)



In [161]:
#Elimnar las columnas que no se utilizarán, en estes caso no creo conveniente tener la columna de "Project" y la de "Case_ID" 
# ya que el proyecto se puede sacar con el grade y el case Id son indiferentes para el analisis de datos posterior
Mutations_df.drop(columns=['Project', 'Case_ID'], inplace=True)

In [162]:
# Función para extraer el tipo de tumor y la especificación del tumor
def extraer_tumor_info(diagnosis):
    if diagnosis == '--' or diagnosis not in ['Oligodendroglioma, NOS', 'Glioblastoma' , 'Mixed glioma', 'Astrocytoma, NOS', 'Astrocytoma, anaplastic', 'Oligodendroglioma, anaplastic']:
        return pd.Series([None, None])
    else:
        parts = diagnosis.split(', ')
        tumor_type = parts[0]
        tumor_specification = parts[1] if len(parts) > 1 else None
        return pd.Series([tumor_type, tumor_specification])

mapeo_tumor = {
    "Oligodendroglioma": 0,
    "Mixed glioma": 1,
    "Astrocytoma": 2,
    "Glioblastoma": 3
}
mapeo_tipo_tumor = {
    None: 0,
    "NOS": 1,
    "anaplastic": 2
}

# Aplicar la función a la columna 'Primary_Diagnosis'
Mutations_df[['Tumor_Type', 'Tumor_Specification']] = Mutations_df['Primary_Diagnosis'].apply(extraer_tumor_info)


# Eliminar filas con 'Primary_Diagnosis' como None o '--'
Mutations_df = Mutations_df.dropna(subset=['Tumor_Type'])

# Aplicar el mapeo a las columnas 'Tumor_Type' y 'Tumor_Specification'
Mutations_df['Tumor_Type'] = Mutations_df['Tumor_Type'].map(mapeo_tumor)
Mutations_df['Tumor_Specification'] = Mutations_df['Tumor_Specification'].map(mapeo_tipo_tumor)


# Eliminar la columna 'Primary_Diagnosis' ya que no se utilizará
Mutations_df.drop(columns=['Primary_Diagnosis'], inplace=True)



In [163]:
# Ordenar las columnas del dataframe para que sea más fácil de leer
column_order = ['Grade', 'Gender', 'Age_at_diagnosis', 'Race', 'Tumor_Type', 'Tumor_Specification' , 'IDH1', 'TP53', 'ATRX', 'PTEN', 'EGFR', 'CIC', 'MUC16', 'PIK3CA', 'NF1', 'PIK3R1', 'FUBP1', 'RB1', 'NOTCH1', 'BCOR', 'CSMD3', 'SMARCA4', 'GRIN2A', 'IDH2', 'FAT4', 'PDGFRA']
Mutations_df = Mutations_df[column_order]

#### Guardamos en un Dataset los datos limpios

In [166]:
# Salvar el dataframe limpio en un nuevo archivo csv
Mutations_df.to_csv("./TCGA_GBM_LGG_Mutations_clean.csv", index=False)

### Tablas de Mapeo

#### [`race_mapping`](command:_github.copilot.openSymbolFromReferences?%5B%22%22%2C%5B%7B%22uri%22%3A%7B%22scheme%22%3A%22vscode-notebook-cell%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2Fc%3A%2FUsers%2FUsuario%2FDesktop%2FMaestria%2F4.-Trimestre%2FMNA%2FMLOPS-A01795088%2FManipulacion_preparacion_datos.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22X32sZmlsZQ%3D%3D%22%7D%2C%22pos%22%3A%7B%22line%22%3A0%2C%22character%22%3A0%7D%7D%5D%2C%22a40ff1a6-404b-4c81-92a6-5a2f575cffbc%22%5D "Go to definition")

| Raza                             | Código |
|----------------------------------|--------|
| -- ó not reported                | 0      |
| white                            | 1      |
| black or african american        | 2      |
| asian                            | 3      |
| american indian or alaska native | 4      |

#### [`mapeo_tumor`](command:_github.copilot.openSymbolFromReferences?%5B%22%22%2C%5B%7B%22uri%22%3A%7B%22scheme%22%3A%22vscode-notebook-cell%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2Fc%3A%2FUsers%2FUsuario%2FDesktop%2FMaestria%2F4.-Trimestre%2FMNA%2FMLOPS-A01795088%2FManipulacion_preparacion_datos.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22X32sZmlsZQ%3D%3D%22%7D%2C%22pos%22%3A%7B%22line%22%3A9%2C%22character%22%3A0%7D%7D%5D%2C%22a40ff1a6-404b-4c81-92a6-5a2f575cffbc%22%5D "Go to definition")

| Tipo de Tumor       | Código |
|---------------------|--------|
| Oligodendroglioma   | 0      |
| Mixed glioma        | 1      |
| Astrocytoma         | 2      |
| Glioblastoma        | 3      |

#### [`mapeo_tipo_tumor`](command:_github.copilot.openSymbolFromReferences?%5B%22%22%2C%5B%7B%22uri%22%3A%7B%22scheme%22%3A%22vscode-notebook-cell%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2Fc%3A%2FUsers%2FUsuario%2FDesktop%2FMaestria%2F4.-Trimestre%2FMNA%2FMLOPS-A01795088%2FManipulacion_preparacion_datos.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22X32sZmlsZQ%3D%3D%22%7D%2C%22pos%22%3A%7B%22line%22%3A15%2C%22character%22%3A0%7D%7D%5D%2C%22a40ff1a6-404b-4c81-92a6-5a2f575cffbc%22%5D "Go to definition")

| Especificación del Tumor | Código |
|--------------------------|--------|
| None                     | 0      |
| NOS                      | 1      |
| anaplastic               | 2      |